# This Notebook will be for writing codes for basic ML like loss functions, optimization functions, etc.. 

### SVM Loss 

Required:
1. Weight matrix
2. Sample input data
3. bias vector

Methods:
1. Loss calculate:

    a. Non - vectorized
    b. Half - Vectorized
    c. Full - Vectorized

In [80]:
import numpy as np

In [81]:
def Debug(desc, value):
    print "\n" + desc +  "\n"
    print value

In [111]:
# Inputs: 
# @y : 1 x N (ground truth labels)
# @wts : C x D (class x feature dim)
# @datapts : => D x N (features x data points)
# @bias : C x 1

# output:
# loss : SVM loss

# TODO: 
# 1. bias can be incorporated inside wts itself by creating one more column vector and corresponding change in datapts.

def svm_loss(wts, datapts, y, bias):
    # TODO: check dim of matrices
    scores = np.add((np.dot(wts, datapts)), np.repeat(bias[:, np.newaxis], y.size, 1));  # bias will be broadcasted
    Debug("Scores: ", scores)
    
    loss_vec = np.zeros(y.size)
    delta = 1.0
    
    ## No vectorization ##
    '''
    # loop for each data point
    for i in range(y.size):
        loss_i = 0;
        correct_label = y[i]
        correct_class_score = scores[correct_label, i]

        # loop for number of classes
        for j in range(scores.shape[0]):
            if(correct_label != j):
                hinge_loss_i_j = max(scores[j, i] - correct_class_score + delta, 0)
                loss_i += hinge_loss_i_j       
        
        loss_vec[i] = loss_i
    '''
    
    ## Half - vectorized  ##
    '''
    for i in range(y.size):
        loss_i = 0;
        correct_class_score = scores[y[i], i]
                
        loss_margins = np.maximum(0, scores[:, i] - correct_class_score + delta)
        loss_margins[y[i]] = 0   # for correct class margin is 0
        print "\nLoss Margin: \n"
        print loss_margins
        
        loss_i = np.sum(loss_margins)        
        
        loss_vec[i] = loss_i
    '''
    
    ##  FULL VECTORIZED ##
    ''
    #scores_mat = np.repeat(scores[:, np.newaxis], scores.size, axis=1)
    #print "\nScores Repeated: \n"
    #print scores_mat
    
    correct_score_mat = scores[y, np.arange(scores.shape[1])]   # 1 x N
    Debug("Correct Label Scores:", correct_score_mat)
    
    loss_margin = np.maximum(scores - correct_score_mat + delta, 0)
    loss_margin[y, np.arange(loss_margin.shape[1])] = 0     # Making loss of actual labels 0
    Debug("Loss Margin: ", loss_margin)
    
    loss_vec = np.sum(loss_margin, axis=0)
    ''
    
    full_loss = np.sum(loss_vec) / loss_vec.size
    
    return full_loss

In [112]:
def get_sample_data():
    #wts = np.random.random((3,4))
    wts = np.array([ [  1,   5,   9,   9],
                     [  6,   3,   3,   6],
                     [  7,   5,   8,   9]
                    ], dtype='f')
    
    # TODO: Center the data
    datapts = np.array([[2, 1, 4, 3],
                       [5, 8, 2, 9],
                       [11, 4, 1, 6],
                       [7, 3, 6, 5]
                       ], dtype='f')
    
    y = np.array([1, 0, 2, 1])
    
    bias = np.random.random(wts.shape[0])
    
    return wts, datapts, bias, y

In [113]:
def test_svm_loss():
    wts, datapts, bias, y = get_sample_data()
    Debug("Weights:", wts)
    
    Debug("Data:", datapts)
    
    loss = svm_loss(wts, datapts, y, bias)
    Debug("Loss:", loss)


In [114]:
test_svm_loss()


Weights:

[[ 1.  5.  9.  9.]
 [ 6.  3.  3.  6.]
 [ 7.  5.  8.  9.]]

Data:

[[  2.   1.   4.   3.]
 [  5.   8.   2.   9.]
 [ 11.   4.   1.   6.]
 [  7.   3.   6.   5.]]

Scores: 

[[ 189.80691509  104.80691509   77.80691509  147.80691509]
 [ 102.56272565   60.56272565   69.56272565   93.56272565]
 [ 190.05670762  106.05670762  100.05670762  159.05670762]]

Correct Label Scores:

[ 102.56272565  104.80691509  100.05670762   93.56272565]

Loss Margin: 

[[ 88.24418944   0.           0.          55.24418944]
 [  0.           0.           0.           0.        ]
 [ 88.49398198   2.24979254   0.          66.49398198]]

Loss:

75.1815338413


In [118]:
# TODO:
# 1. Take care of numerical stability for high scores
# 2. Regularization

def softmax_loss(wts, datapts, y, bias):
    # TODO: check dim of matrices
    scores = np.add((np.dot(wts, datapts)), np.repeat(bias[:, np.newaxis], y.size, 1));  # bias will be broadcasted
    Debug("Scores: ", scores)
    
    correct_scores = scores[y, np.arange(scores.shape[1])]
    Debug("Correct Scores:", correct_scores)

    correct_scores_exp = np.exp(correct_scores)
    
    # TODO: it is better to center the scores for each data point before applying exp and taking sum. 
    # This will avoid numerical bloating
    scores_exp = np.exp(scores)    
    scores_exp_sum = np.sum(scores_exp, axis=0)
    Debug("Exponential Scores Sum:", scores_exp_sum)
    
    prob = correct_scores_exp / scores_exp_sum
    Debug("Softmax output: ", prob)
    
    log_prob = np.log(prob)
    Debug("Log prob: ", log_prob)
    
    loss = -1 * np.sum(log_prob) / y.size
    
    return loss

In [119]:
def test_softmax_loss():
    wts, datapts, bias, y = get_sample_data()
    Debug("Weights:", wts)
    
    Debug("Data:", datapts)
    
    loss = softmax_loss(wts, datapts, y, bias)
    Debug("Loss:", loss)

In [122]:
test_softmax_loss()


Weights:

[[ 1.  5.  9.  9.]
 [ 6.  3.  3.  6.]
 [ 7.  5.  8.  9.]]

Data:

[[  2.   1.   4.   3.]
 [  5.   8.   2.   9.]
 [ 11.   4.   1.   6.]
 [  7.   3.   6.   5.]]

Scores: 

[[ 189.39185846  104.39185846   77.39185846  147.39185846]
 [ 102.19897635   60.19897635   69.19897635   93.19897635]
 [ 190.56834248  106.56834248  100.56834248  159.56834248]]

Correct Scores:

[ 102.19897635  104.39185846  100.56834248   93.19897635]

Exponential Scores Sum:

[  7.57717233e+82   2.13161998e+46   4.74543681e+43   1.99366812e+69]

Softmax output: 

[  3.19848500e-39   1.01882198e-01   1.00000000e+00   1.50019696e-29]

Log prob: 

[ -8.86381414e+01  -2.28393806e+00  -8.60401750e-11  -6.63693713e+01]

Loss:

39.3228626776
